In [1]:
%load_ext autoreload

In [2]:
import os, sys

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow.keras as keras

In [4]:
BASE_PATH = os.path.join(os.getcwd(), "..", "..", "..")
MODULES_PATH = os.path.join(BASE_PATH, "modules")
DATASET_PATH = os.path.join(BASE_PATH, "datasets")

In [5]:
sys.path.append(MODULES_PATH)

In [6]:
from active_learning import Config, AcquisitionFunction, Pool
from wrapper import McDropout, MomentPropagation
from models import setup_growth, fchollet_cnn
from data import BenchmarkData, DataSetType

In [7]:
mnist_path = os.path.join(DATASET_PATH, "mnist")
dataset = BenchmarkData(DataSetType.MNIST, mnist_path)

In [8]:
setup_growth()

1 Physical GPU's,  1 Logical GPU's


In [9]:
base_model = fchollet_cnn(output=10)

In [10]:
seed = 90231
np.random.seed(seed)
tf.random.set_seed(seed)

### Split data

In [11]:
x_train, x_test, y_train, y_test = train_test_split(dataset.inputs, dataset.targets, test_size=10000)

In [12]:
len(x_train)

60000

In [13]:
pool = Pool(x_train, y_train)

### Define Models

In [14]:
%autoreload 2
fit_params = {"epochs": 2, "batch_size": 10}
compile_params = {"optimizer": "adam", "loss": "sparse_categorical_crossentropy", "metrics": [keras.metrics.SparseCategoricalAccuracy()]}

# Define MC Dropout model
mc_model = McDropout(base_model, config=Config(
    fit=fit_params,
    query={"sample_size": 25},
    evaluate={"sample_size": 25}
))
mc_model.compile(**compile_params)

# Define Moment Propagation model
mp_model = MomentPropagation(base_model, config=Config(
    fit=fit_params
))
mp_model.compile(**compile_params)

In [15]:
mc_model.fit(x_train, y_train)

Epoch 1/2
6000/6000 [==============================] - 16s 3ms/step - loss: 0.1905 - sparse_categorical_accuracy: 0.9426
Epoch 2/2
6000/6000 [==============================] - 16s 3ms/step - loss: 0.0848 - sparse_categorical_accuracy: 0.9750


In [16]:
mp_model.fit(x_train, y_train)

Epoch 1/2
6000/6000 [==============================] - 18s 3ms/step - loss: 0.0648 - sparse_categorical_accuracy: 0.9808
Epoch 2/2
6000/6000 [==============================] - 17s 3ms/step - loss: 0.0522 - sparse_categorical_accuracy: 0.9844


In [17]:
mc_model.evaluate(x_test[:10], y_test[:10])

{'loss': 0.0017364342, 'accuracy': 1.0}

In [18]:
mp_model.evaluate(x_test[:10], y_test[:10])

{'loss': 0.0017244013, 'accuracy': 1.0}

### Try BALD acquisition

In [19]:
mc_bald = mc_model.get_query_fn("bald")
mc_bald(x_train[:100], sample_size=100)

array([2.69152224e-06, 1.74771436e-03, 2.94139609e-05, 3.97236057e-04,
       9.89236636e-04, 4.43089530e-02, 9.76431370e-02, 1.03059769e-01,
       8.14773981e-03, 1.31806650e-04, 4.14963029e-02, 2.22752690e-01,
       1.76340109e-05, 1.08567276e-03, 1.16066545e-01, 6.72197435e-03,
       5.49966544e-02, 4.69741644e-04, 1.17583806e-03, 1.10080272e-01,
       1.22740777e-04, 7.47246295e-03, 2.04886030e-02, 2.13370193e-04,
       5.27996477e-03, 5.85220987e-04, 1.05642444e-02, 6.58327062e-03,
       2.32467744e-02, 8.12176429e-03, 2.72173472e-02, 4.75801416e-02,
       1.17077790e-02, 2.81655230e-05, 2.70120144e-01, 4.84878607e-02,
       9.69808176e-03, 6.03630953e-03, 6.98673911e-03, 6.20925566e-04,
       3.59753706e-03, 5.68607915e-03, 8.25164956e-04, 3.34565993e-05,
       3.08435500e-01, 1.11485994e-03, 7.64455944e-02, 2.03689560e-05,
       1.98385969e-01, 1.24464370e-02, 1.58697873e-01, 4.70032683e-06,
       1.45048097e-01, 1.52271660e-02, 1.68758864e-03, 6.54511678e-04,
      

In [20]:
mp_bald = mp_model.get_query_fn("bald")
mp_bald(x_train[:100], num_samples=100)

array([ 1.15527288e-04, -1.35166444e-03,  1.11645479e-03,  1.43843769e-03,
        3.14362407e-03,  2.17596606e-02,  1.68430793e-02,  3.60616119e-02,
        6.30286502e-04,  1.03669056e-03,  6.19976894e-02,  2.49540101e-01,
       -4.59969021e-04,  6.74851138e-03,  1.27086419e-01,  4.11311535e-03,
        7.84893230e-02,  2.11568496e-03, -3.44948374e-04,  4.65166584e-02,
       -3.24993361e-03, -9.67022050e-04,  3.13500409e-03,  1.64719784e-03,
        1.00264818e-02,  3.57045166e-03,  1.97598255e-02, -1.04861740e-03,
        2.36856736e-02,  9.15133327e-03,  6.22308648e-02,  2.88434759e-02,
       -2.79475650e-03, -4.01009707e-04,  2.99667917e-01,  6.65881477e-02,
        8.55774188e-03,  5.23705754e-03,  1.78617747e-02,  4.02966475e-03,
       -8.44746182e-03,  5.06869365e-03,  1.33233388e-02, -3.32076931e-04,
        2.46791420e-01,  1.16539059e-02,  6.70872000e-02,  7.96762257e-04,
        2.46243354e-01,  8.33244982e-03,  1.48968871e-01, -1.50657531e-04,
        4.13716185e-02,  

## Difference Monte Carlo approximation and analytical

In [21]:
# Define Moment Propagation model
fit_params = {"epochs": 10, "batch_size": 500}
mp_model = MomentPropagation(base_model, config=Config(
    fit=fit_params
))
mp_model.compile(**compile_params)

In [22]:
mp_model.fit(x_train, y_train)

Epoch 1/10
120/120 [==============================] - 3s 21ms/step - loss: 0.0288 - sparse_categorical_accuracy: 0.9877
Epoch 2/10
120/120 [==============================] - 3s 21ms/step - loss: 0.0240 - sparse_categorical_accuracy: 0.9927
Epoch 3/10
120/120 [==============================] - 3s 21ms/step - loss: 0.0207 - sparse_categorical_accuracy: 0.9938
Epoch 4/10
120/120 [==============================] - 3s 21ms/step - loss: 0.0170 - sparse_categorical_accuracy: 0.9950
Epoch 5/10
120/120 [==============================] - 3s 21ms/step - loss: 0.0158 - sparse_categorical_accuracy: 0.9953
Epoch 6/10
120/120 [==============================] - 3s 22ms/step - loss: 0.0136 - sparse_categorical_accuracy: 0.9956
Epoch 7/10
120/120 [==============================] - 3s 22ms/step - loss: 0.0127 - sparse_categorical_accuracy: 0.9960
Epoch 8/10
120/120 [==============================] - 3s 22ms/step - loss: 0.0112 - sparse_categorical_accuracy: 0.9962
Epoch 9/10
120/120 [====================

In [23]:
mp_model.evaluate(x_test, y_test)

{'loss': 0.029292136, 'accuracy': 0.9913}

In [24]:
exp, var = mp_model(x_train[:100])

In [25]:
exp_s, var_s = MP.Gaussian_Softmax(exp, var)

NameError: name 'MP' is not defined

In [ ]:
def evaluate(prob, real_targets):
    """
        Evaluate accuracy of predictions.
        
        Parameters:
            prob (numpy.ndarray): Probabilities given by estimator. With optional sample dimension.
            real_targets (numpy.ndarray): The real targets
            
        Returns:
            (float) the accuracy of the estimator
    """
    
    if len(prob.shape) == 2:
        return np.mean(np.argmax(prob, axis=-1)==real_targets, axis=0)
    return np.mean(np.argmax(np.mean(sampled_datapoints, axis=0), axis=1)==real_targets)

In [ ]:
exp_shape = list(exp.shape)
sample_sizes = list(np.linspace(0, 100, 101).astype(int))
acc = []
for size in sample_sizes:
    
    final_shape = tuple([size] + exp_shape)
    sampled_datapoints = norm(exp, var).rvs(size=final_shape)
    softmax_output = tf.keras.activations.softmax(tf.convert_to_tensor(sampled_datapoints))
    sample_acc = evaluate(softmax_output, y_train[:100])

    acc.append(sample_acc)

In [ ]:
real_acc = evaluate(exp_s, y_train[:100])

In [ ]:
fig = plt.figure()
plt.plot(sample_sizes, acc, label="Analytical acc.")
plt.plot(sample_sizes, [real_acc]*len(sample_sizes), label="Monte Carlo acc.")

plt.legend()
plt.grid()
plt.xlabel("Number of Samples")
plt.ylabel("Accuracy")
plt.title("Estimator Accuracy")
plt.show()

In [ ]:
approx_entropy = np.sum(exp_s*np.log(exp_s+.001), axis=1)
approx_entropy

In [ ]:
final_shape = tuple([100] + exp_shape)
sampled_datapoints = norm(exp, var).rvs(size=final_shape)
softmax_output = tf.keras.activations.softmax(tf.convert_to_tensor(sampled_datapoints))
sample_acc = evaluate(softmax_output, y_train[:100])

In [ ]:
sampled_datapoints.shape

In [ ]:
sample_probs = tf.keras.activations.softmax(tf.convert_to_tensor(sampled_datapoints))

# mean_probs = np.mean(sample_probs, axis=0)
# sample_entropy = np.sum(mean_probs*np.log(mean_probs+.001), axis=-1)

sample_entropy = np.sum(sample_probs*np.log(sample_probs+.001), axis=-1)
sample_entropy = np.sum(sample_entropy, axis=0)/len(sample_entropy)

In [ ]:
sample_entropy.shape

In [ ]:
plt.plot(sample_entropy, label="Sample entropy")
plt.plot(approx_entropy, label="Approx. entropy")
plt.legend()
plt.plot()

## Condfidence interval

In [ ]:
# https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data
# https://blog.finxter.com/how-to-plot-the-confidence-interval-in-python/
import scipy.stats as st

In [ ]:
sample_probs = tf.keras.activations.softmax(tf.convert_to_tensor(sampled_datapoints))

In [ ]:
res = st.t.interval(0.95, len(sample_probs)-1, loc=np.mean(sample_probs, axis=0), scale=st.sem(sample_probs))